In [ ]:
import sqlite3 as sql
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
from collections import Counter 
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

In [ ]:
# Creer une connection d'objets,
# Creer une nouvelle db s'elle n'existe pas 
# connectez-le, s'il existe, connectez-vous.
connection = sql.connect('spotifyprojet.db')

# Créer un objet curseur
curs = connection.cursor()

In [ ]:
#Charger les données des fichier "artiste.CSV" et "chansons.CSV" dans Pandas DataFrame
artistes = pd.read_csv(r'C:\Users\Simplon\Documents\Spotify\artistes.csv', encoding="UTF8") 
chansons = pd.read_csv(r'C:\Users\Simplon\Documents\Spotify\chansons.csv', encoding="UTF8") 

In [ ]:
# Écrire les données dans une table db SQlite
artistes.to_sql('spotifyprojet', connection, if_exists='replace', index=False)
chansons.to_sql('spotifyprojet', connection, if_exists='replace', index=False)

In [ ]:
connection.close()

In [ ]:
#nettoyer la colonne artists et id artists dans la table chansons

chansons["artists"]=chansons["artists"].str.replace("[", "")
chansons["artists"]=chansons["artists"].str.replace("]", "")
chansons["artists"]=chansons["artists"].str.replace("'", "")

chansons["id_artists"]=chansons["id_artists"].str.replace("[", "")
chansons["id_artists"]=chansons["id_artists"].str.replace("]", "")
chansons["id_artists"]=chansons["id_artists"].str.replace("'", "")

In [ ]:
# Verifier s'il manque des donnée artistes 
a=artistes.isna().sum().sum() 
print(a)

In [ ]:
# Verifier s'il manque des donnée chansons
b=chansons.isna().sum().sum() 
print(b)

In [ ]:
# enelever les colonne avec des valeur NAN
chansons.dropna(inplace = True)
artistes.dropna(inplace = True)
# dropna(inplace = True) sans declarer chansons_na 

In [ ]:
#vérifier si les NAN sont bien enlevés
chansons.describe()

In [ ]:
#Convert Milli secs duration into minutes
chansons['duration_min'] = chansons['duration_ms']/60000
chansons['duration_min'] = chansons['duration_min'].round(2)

In [ ]:
def to_year(x):
    result = []
    x.to_string()
    for i in x:
        result.append(int(i[:4]))
    return result

chansons['year'] = to_year(chansons.release_date)

In [ ]:
chansons.drop(columns=['duration_ms'], inplace = True)
chansons

In [ ]:
# enelver les colonnes name id id_artists artists release_date
chansons_num = chansons.drop(columns=['name','id','id_artists','artists','release_date'])
z = chansons_num.values

In [ ]:
# normaliser ma table chansons

min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(z)
chansons1 = pd.DataFrame(x_scaled)

In [ ]:
#tracer un tableau de corrélation 

corr = chansons_num.corr() 
plt.figure(figsize=(20,10))
sns.set(style="whitegrid")
sns.heatmap(corr,annot=True,cmap='BrBG_r',annot_kws={"fontsize":12})

In [ ]:
#créer table de nb de chansons par artists 
song_per_artists = chansons.value_counts(subset=['artists']).reset_index(name="count")
song_per_artists.head(20)

In [ ]:
#créer table de nb de chansons par artists 
song_per_artists = chansons.value_counts(subset=['id_artists']).reset_index(name="count")
song_per_artists.head(20)

In [ ]:
song_per_artists.describe()

In [ ]:
# Figure nb de chanson/artists
fig = plt.figure(figsize=(12,6))
echelle = fig.add_subplot(1,1,1)
song_per_artists['count'].plot(kind = 'hist',bins=700)
echelle.set_ylim([0,8000])
echelle.set_xlim([0,150])
plt.title('nb chansons/artists')
plt.show()

In [ ]:
# Figure popularity
fig = plt.figure(figsize=(18,8))
echelle = fig.add_subplot(1,1,1)
chansons["popularity"].plot(kind = 'hist',bins=100)
plt.title('popularity')
plt.xlabel('popularity des chansons')
plt.ylabel('nombre de chansons')
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,8))
echelle = fig.add_subplot(1,1,1)
chansons["duration_min"].plot(kind = 'hist',bins=1000)
echelle.set_xlim([0,13])
plt.title('temps moyen des morceaux')
plt.xlabel('durée des chansons')
plt.ylabel('nombre de chansons')
plt.show()

In [ ]:
fig = plt.figure(figsize=(18,8))
echelle = fig.add_subplot(1,1,1)
chansons["tempo"].plot(kind = 'hist',bins=1000)
echelle.set_xlim([50,220])
plt.title('tempo')
plt.xlabel('tempo des chansons')
plt.ylabel('nombre de chansons')
plt.show()

In [ ]:
L = chansons["energy"]
E = chansons["loudness"]
plt.scatter(L, E)
plt.show()

In [ ]:
A = chansons["valence"]
B = chansons["danceability"]
plt.scatter(A, B)
plt.show()

In [ ]:
L = chansons["energy"]
E = chansons["acousticness"]
plt.scatter(L, E)
plt.show()

In [ ]:
columns = ["acousticness","danceability","energy","speechiness","liveness","valence"]
fig = plt.figure(figsize=(30,30))
echelle = fig.add_subplot(1,1,1)
echelle.set_xlim([1940,2020])

for c in columns:
    x = chansons.groupby('year')[c].mean()
    sns.lineplot(x.index,x,label=c,lw=4)

plt.title('Audio characteristics over the years', fontsize=40)
plt.xlabel('Year',fontsize=30)
plt.ylabel('Characteristics',fontsize=30)
plt.legend(fancybox=True, framealpha=1, shadow=True, borderpad=1, prop={'size': 20}, loc = 'upper right')
plt.show()

In [ ]:
# trier les données du fichiers artistes pour le parametre popularity en ordre decroissant
artistes.sort_values('popularity', ascending = False, inplace = True)

In [ ]:
# trier les données du fichiers chansons pour le parametre popularity en ordre decroissant
chansons.sort_values('tempo', ascending = False, inplace = True)

In [ ]:
#trier les données du fichiers artistes pour le parametre followers en ordre decroissant
artistes.sort_values('followers', ascending = False, inplace = True)

In [ ]:
#afficher les 10 chansons les plus populaires à base da la popularity
chansons_popu =chansons.groupby("name")['tempo'].mean().sort_values(ascending=True).head(4000)
print(chansons_popu)

In [ ]:
#afficher les 10 artistes les plus populaires à base da la popularity
artistes_popu =artistes.groupby("name")['popularity'].mean().sort_values(ascending=False).head(10)
print(artistes_popu)

In [ ]:
#afficher les 10 chansons les plus suivi à base de followers
artistes_follow =artistes.groupby("name")['followers'].mean().sort_values(ascending=False)
print(artistes_follow)

In [ ]:
#fusionner mes deux tables 
fusion = chansons.merge(artistes, how='left', left_on = 'id_artists', right_on='id' )
fusion.info()

In [ ]:
fusion.to_csv (r'E:\table_fusion.csv')

In [ ]:
q = fusion.isna().sum()
print(q)

In [ ]:
# Recup chansons id_y = na 
fusion_na = fusion[fusion["id_y"].isna()]
fusion_na

In [ ]:
#drop les lignes qui manquent entre les deux table 
fusion.dropna(inplace=True)
fusion.info()

In [ ]:
# visualiser la corrélation de table fusion
corr = fusion.corr()
plt.figure(figsize=(20,10))
sns.set(style="whitegrid")
sns.heatmap(corr,annot=True,cmap='BrBG_r',annot_kws={"fontsize":12})


In [ ]:
x = fusion.drop(columns=['name_x','name_y','id_x','id_y','id_artists','artists','release_date','genres'])

In [ ]:
#date release à nettoyer 
y = pd.DataFrame(fusion.iloc[:,2:4].values)

In [ ]:
scaler = preprocessing.StandardScaler()
Y_scaled= pd.DataFrame(scaler.fit_transform(y),columns=y.columns)

In [ ]:
X_scaled = pd.DataFrame(scaler.fit_transform(x),columns=x.columns)

y1 = Y_scaled.iloc[:,0].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( x, y1, test_size=0.3, random_state=0)

In [ ]:
model = LinearRegression()

model.fit(X_train, y_train)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
model.score(y_test, y_pred)

### 